## Australia Data Science Jobs - Exploratory Data Analysis

## Set up notebook

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from fuzzywuzzy import process
pd.set_option('display.max_rows', None)

## read input csv and convert to datasets

In [ ]:
file1 = r'/kaggle/input/australia-data-science-jobs/AustraliaDataScienceJobs.csv'
file2 = r'/kaggle/input/australia-data-science-jobs/AustraliaDataScienceJob2.csv'

# Re-importing datasets and checking for data issues
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Combine the datasets
df = pd.concat([df1, df2])

# Fuzzy match job titles

In [ ]:
# Checking for non-string values in 'Job Title' column
df['Job Title'] = df['Job Title'].astype(str)

# Define the categories and their possible titles
categories = {
    "Data Analyst": ["data analyst"],
    "Data Engineer": ["data engineer"],
    "Data Scientist": ["data scientist"]
}

# Function to classify job title
def classify_job_title(title):
    best_match = None
    highest_score = 0
    
    for category, titles in categories.items():
        match, score = process.extractOne(title, titles)
        if score > highest_score:
            highest_score = score
            best_match = category
    
    return best_match if highest_score >= 70 else "Other Analyst"

df['Job Category'] = df['Job Title'].apply(classify_job_title)

# Count and Ave. Salary by Job

In [ ]:
# Calculate average salary and count per job category
average_salary_count = df.groupby('Job Category')['High Estimate'].agg(['mean', 'count']).reset_index()
average_salary_count.columns = ['Job Category', 'Average Salary', 'Job Count']

# Format the output to make it prettier
average_salary_count['Average Salary'] = average_salary_count['Average Salary'].apply(lambda x: f"${x:,.2f}")

In [ ]:
average_salary_count

# Skills by job

In [ ]:
# Extract skill columns
skill_columns = [col for col in df.columns if col.endswith('_yn')]

# Calculate skill percentages per job category
skill_percentages = df.groupby('Job Category')[skill_columns].mean() * 100
skill_percentages = skill_percentages.reset_index()

# Rename columns
skill_percentages.columns = ['Job Category'] + [col.replace('_yn', '') for col in skill_percentages.columns if col != 'Job Category']

# Round percentages to 2 decimal places
skill_percentages = skill_percentages.round(2)

# Transpose the data to have job titles as columns and skills as rows
skill_percentages_pivot = skill_percentages.set_index('Job Category').transpose()

# Pct of time skill is mentioned per job

In [ ]:
# Display the result
skill_percentages_pivot

# Salary and Job count by Company 

In [ ]:
# Calculate average salary and count per company
company_salary_count = df.groupby('Company')['High Estimate'].agg(['mean', 'count']).reset_index()
company_salary_count.columns = ['Company', 'Average Salary', 'Job Count']

# Format the salary and sort by job count
company_salary_count['Average Salary'] = company_salary_count['Average Salary'].apply(lambda x: f"${int(x):,}")
company_salary_count = company_salary_count.sort_values(by='Job Count', ascending=False).reset_index(drop=True)

# Display the result
company_salary_count

# Ave. Salary by Job and Company

In [ ]:
# Create a pivot table with job titles as columns, companies as rows, and average salary as values
pivot_table = df.pivot_table(index='Company', columns='Job Category', values='High Estimate', aggfunc='mean')

# Format the salaries to integer with commas
pivot_table = pivot_table.applymap(lambda x: f"${int(x):,}" if pd.notnull(x) else '-')

# Display the result
pivot_table

# Job Count by Job and Company

In [ ]:
# Create a pivot table with job titles as columns, companies as rows, and job count as values
pivot_table_counts = df.pivot_table(index='Company', columns='Job Category', values='Job Title', aggfunc='count', fill_value=0)

# Display the result
pivot_table_counts